# From spreadsheet to Kobo
---

![](https://media.giphy.com/media/xUNd9KMFMPtn5reCiI/giphy.gif)

Sometimes you might want to import data from a spreadsheet into the Kobo database. Maybe you've exported your original data into a spreadsheet, done some data-cleaning and now you want it back. If that's the case, you may want to checkout the [bulk updating feature](https://support.kobotoolbox.org/howto_edit_multiple_submissions.html) now included in the UI. Whatever the case, perhaps the following tutorial may help — or at least guide you in a right direction.

## Get things setup
---

You may need to `pip install` some necessary packages to perform some functions depending on your local setup. I'm currently using the [`jupyter/datascience-notebook`](https://registry.hub.docker.com/r/jupyter/datascience-notebook/#!) Docker image. If you have Docker installed locally, you can do the same by running the following command and pasting the generated URL into your browser:

```bash
docker run -p 8888:8888 -v ~/notebooks:/home/jovyan jupyter/datascience-notebook
```

Once that's out the way, let's go ahead and import all that we need.

### Install any necessary packages ⬇️

In [1]:
!pip install openpyxl

### Import whatever you need 👇

In [2]:
import io
import requests
import uuid
from datetime import datetime
from random import choice, randint, sample
from time import sleep
from xml.etree import ElementTree as ET

import pandas as pd
import pytz

### Set up some helpful constants that we'll use below 🤘

In [21]:
# You can find this by navigating to https://kc.kobotoolbox.org/token
TOKEN = '<TOKEN>'

In [4]:
KC_URL = 'https://kc.kobotoolbox.org'
KF_URL = 'https://kf.kobotoolbox.org'

ASSET_UID = 'aLmmfWSAUNamKwiSTcVSis'

DATA_URL = f'{KF_URL}/api/v2/assets/{ASSET_UID}/data'
XML_URL = f'{DATA_URL}.xml'
SUMISSION_URL = f'{KC_URL}/api/v1/submissions'

HEADERS = {
    'Authorization': f'Token {TOKEN}'
}
PARAMS = {
    'format': 'json'
}

## Let's create some random submission data 🎉

![](https://media.giphy.com/media/xT9C25UNTwfZuk85WP/giphy.gif)

For the purposes of demonstration, let's generate a list of submissions with random names and random choices from the available list of pizza toppings: cheese, pepperoni and saussage.

Those submissions will then be exported into an Excel file so that we have what we need to continue with the walkthrough of submitting data from an Excel spreadsheet.

In [5]:
names_url = 'https://raw.githubusercontent.com/dominictarr/random-name/master/first-names.txt'
names = requests.get(names_url).content.decode().split()
topings = ['cheese', 'pepperoni', 'sausage']

In [6]:
NUBER_OF_SUBMISSIONS = 20
submissions = []
for _ in range(NUBER_OF_SUBMISSIONS):
    submissions.append(
        {
            'name': choice(names),
            'favorite_toppings': ' '.join(sample(topings, randint(1, 3)))
        }
    )

In [7]:
submissions[:10]

[{'name': 'Bunni', 'favorite_toppings': 'sausage'},
 {'name': 'Letisha', 'favorite_toppings': 'sausage pepperoni'},
 {'name': 'Sara', 'favorite_toppings': 'sausage'},
 {'name': 'Michelina', 'favorite_toppings': 'sausage'},
 {'name': 'Rachelle', 'favorite_toppings': 'pepperoni sausage cheese'},
 {'name': 'Louella', 'favorite_toppings': 'sausage pepperoni'},
 {'name': 'Gus', 'favorite_toppings': 'pepperoni cheese sausage'},
 {'name': 'Cacilia', 'favorite_toppings': 'pepperoni cheese sausage'},
 {'name': 'Mae', 'favorite_toppings': 'pepperoni'},
 {'name': 'Margit', 'favorite_toppings': 'cheese pepperoni'}]

In [8]:
subs = pd.DataFrame(submissions)

In [9]:
subs.head()

,name,favorite_toppings
0,Bunni,sausage
1,Letisha,sausage pepperoni
2,Sara,sausage
3,Michelina,sausage
4,Rachelle,pepperoni sausage cheese


Export that data into a spreadsheet!

In [10]:
subs.to_excel('submissions.xlsx', index=False)

## Moving on 🚶‍♂️
---

Now that we've got some submission data in an Excel spreadsheet, we can move on with the how to submit it.

![](https://media.giphy.com/media/Y0arChV1qGnQ3NWX9C/giphy.gif)

In [11]:
subs = pd.read_excel('submissions.xlsx')

In [12]:
subs.head()

,name,favorite_toppings
0,Bunni,sausage
1,Letisha,sausage pepperoni
2,Sara,sausage
3,Michelina,sausage
4,Rachelle,pepperoni sausage cheese


### Now the fun begins 🕺

![](https://media.giphy.com/media/3oEjHKiDeYrKnjwGFq/giphy.gif)

There are several ways we can go about constructing the XML for each submission, but what may be the simplist is to pull the XML structure of an existing submission (if you don't have one yet, just create a submission through Enketo) and then use that as a template.

So, assuming there is at least one submission already, let's continue:

In [13]:
res = requests.get(url=XML_URL, headers=HEADERS, params=PARAMS)

In [14]:
res.status_code

200

If you take a look at the XML structure, the submissions are nested within `results` and use the `ASSET_UID` as their root.

In [15]:
parsed_xml = ET.fromstring(res.text)
e = parsed_xml.find(f'results/{ASSET_UID}')
template = ET.tostring(e)

In [16]:
print(template.decode())

<aLmmfWSAUNamKwiSTcVSis id="aLmmfWSAUNamKwiSTcVSis" version="1 (2021-03-25 21:50:30)">
          <formhub>
            <uuid>8d4cc6d68c004f399af90084baefa99d</uuid>
          </formhub>
          <name>Josh</name>
          <favorite_toppings>cheese pepperoni sausage</favorite_toppings>
          <__version__>vA6sVYHjMET6Dpet8PLqks</__version__>
          <meta>
            <instanceID>uuid:1341837c-fbd3-4400-af87-738e5b93a5eb</instanceID>
          </meta>
        </aLmmfWSAUNamKwiSTcVSis>


### Time for some hacking 💻

![](https://media.giphy.com/media/wpoLqr5FT1sY0/giphy.gif)

Now that we've got a template, let's go ahead and create some helpful methods to parse and update that template with the data in the Excel spreadsheet.

(If you're interested, most of the process below is adapted from the code in KPI that interfaces with Kobocat [here](https://github.com/kobotoolbox/kpi/blob/d56621b6daced1891bc9fe2661c2aafe9e9a92a4/kpi/deployment_backends/kobocat_backend.py#L641-L722).)

In [17]:
def submit_data(xml_sub: bytes, _uuid: str) -> str:
    """
    Send the XML to kobo!
    """
    file_tuple = (_uuid, io.BytesIO(xml_sub))
    files = {'xml_submission_file': file_tuple}
    res = requests.Request(
        method='POST', url=SUMISSION_URL, files=files, headers=HEADERS
    )
    session = requests.Session()
    res = session.send(res.prepare())
    return res.status_code

def format_openrosa_datetime() -> str:
    """
    This is required to get the correct datetime formatting
    """
    return datetime.now(tz=pytz.UTC).isoformat('T', 'milliseconds')

def update_element_value(e: ET.Element, name: str, value: str) -> None:
    """
    Get or create a node and give it a value
    """
    el = e.find(name)
    if el is None:
        el = ET.SubElement(e, name)
    el.text = value

def create_submissions(data: pd.DataFrame) -> list:
    """
    Take a bunch of submissions and send them off
    """
    all_subs = []
    parsed_xml = ET.fromstring(template)
    for i, row in data.iterrows():
        _now = format_openrosa_datetime()
        _uuid = str(uuid.uuid4())
        
        for item in data.columns:
            update_element_value(parsed_xml, item, row[item])
        
        # We have to update the instanceID, otherwise there'll be issues
        update_element_value(parsed_xml, 'meta/instanceID', f'uuid:{_uuid}')
        
        # Updating the `start` and `end` times is not really necessary, but 
        # probably something you'd want to do
        update_element_value(parsed_xml, 'start', _now)
        update_element_value(parsed_xml, 'end', _now)
        
        all_subs.append(submit_data(ET.tostring(parsed_xml), _uuid))
        
        # If you are submitting a large amount of data, please be mindful that it can
        # overwhelm the servers if sent in a short span of time. Letting it sleep for
        # for a short stint between each upload will be much appreciated
        sleep(0.2)
        
    return all_subs

In [18]:
responses = create_submissions(subs)

Let's do a quick check to see if all the responses were successful:

In [19]:
all(res == 201 for res in responses)

True

If there were a few unsuccessful, let's see how many...

In [20]:
pd.Series(responses).value_counts()

201    20
dtype: int64

## All done 🔥

Check the data table in the Kobo UI and ensure that your data has been successfully submitted.

![](https://media.giphy.com/media/lD76yTC5zxZPG/giphy.gif)